In [38]:
import csv
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from collections import *
import math
from statistics import mode
%load_ext jupyternotify

The jupyternotify extension is already loaded. To reload it, use:
  %reload_ext jupyternotify


In [2]:
canto_data = pd.read_csv('./data/full_cantometrics.csv').drop(columns=['Unnamed: 0'])

In [3]:
# canto_data = pd.read_csv('./output/full_cantometrics.csv')
features = ['line_'+str(i+1)for i in range(37)]
cultures = canto_data['Preferred_name'].unique()
columns = ['Preferred_name', 'soc_id']
for feature in features:
    columns.append(feature)

In [4]:
%%notify -m "final matrix imputed"
final_df = pd.DataFrame(columns = columns)
for culture in cultures:
    modal_profile = dict(canto_data[canto_data['Preferred_name']==culture][features].mode().loc[0])
    modal_profile['Preferred_name'] = culture
    soc_id = canto_data[canto_data['Preferred_name']==culture]['society_id'].reset_index()['society_id'][0]
    modal_profile['soc_id'] = soc_id
    final_df = final_df.append(modal_profile, ignore_index=True)

<IPython.core.display.Javascript object>

## Modal Profiles without multi-codings

In [8]:
downloaded_data = pd.read_csv('./data/downloaded_data.csv')

In [9]:
# Load conversion guide
with open('./data/conversion_guide.json') as f:
  conversion_guide = json.load(f)

df = downloaded_data.drop(columns=['orv_1', 'orv_2', 'ensemble_value_id',
       'ensemble_value_label', 'instrument_value_id',
       'instrument_value_label','Unnamed: 0'])

feature_cols = ["line_"+str(i+1) for i in range(37)]

In [66]:
def encode(line, binary):
    for conversion in conversion_guide[line]:
        if conversion['code']==binary:
            codes = [conversion["original_1"], 
                     conversion["original_2"], 
                     conversion["original_3"]]
            return [code for code in codes if code]

In [86]:
grouped = df.groupby('society_id')
final_cols = ['soc_id', 'lat','lng', 'Preferred_name','line_1', 'line_2', 'line_3',
       'line_4', 'line_5', 'line_6', 'line_7', 'line_8', 'line_9', 'line_10',
       'line_11', 'line_12', 'line_13', 'line_14', 'line_15', 'line_16',
       'line_17', 'line_18', 'line_19', 'line_20', 'line_21', 'line_22',
       'line_23', 'line_24', 'line_25', 'line_26', 'line_27', 'line_28',
       'line_29', 'line_30', 'line_31', 'line_32', 'line_33', 'line_34',
       'line_35', 'line_36', 'line_37']
final_df = pd.DataFrame(columns=final_cols)
for name, group in grouped:
    modes = {}
    for line in feature_cols:
        line_encodings = []
        for item in group[line]:
            codes = []
            codes = encode(line, item)
            line_encodings.extend(codes)
            try:
                line_mode = pow(2, mode(line_encodings))
            except:
                line_mode = 'NaN'
        modes[line] = line_mode
    modes["soc_id"] = name
    modes["Preferred_name"] = mode(group["Preferred_name"])
    final_df = final_df.append(modes, ignore_index=True)

In [88]:
metadata = pd.read_csv('./data/metadata.csv')

In [89]:
for i, row in final_df.iterrows():
    culture = row['Preferred_name']
    try:
        lat = metadata[metadata['Preferred_name']==culture].reset_index()['Local_latitude'][0]
        lng = metadata[metadata['Preferred_name']==culture].reset_index()['Local_longitude'][0]
    except:
        final_df.drop([i])
    final_df.loc[i, ['lat']] = lat
    final_df.loc[i, ['lng']] = lng

In [72]:
final_df.to_csv('./output/modal_profiles.csv')

In [87]:
final_df

,soc_id,lat,lng,Preferred_name,line_1,line_2,line_3,line_4,line_5,line_6,...,line_28,line_29,line_30,line_31,line_32,line_33,line_34,line_35,line_36,line_37
0,10000,NaN,NaN,'Are'are,8192,2,64,8192,1024,1024,...,32,128,128,8192,1024,256,128,8192,8192,1024
1,10027,NaN,NaN,Abaluhya,256,4,2048,128,128,1024,...,8192,8192,128,8192,1024,256,8192,16,1024,1024
2,10033,NaN,NaN,Abelam,256,2,2,128,128,1024,...,2,8192,128,8192,128,64,1024,16,1024,8192
3,10055,NaN,NaN,Abkhazia,512,8,64,8192,8192,8192,...,8192,8192,128,8192,1024,1024,8192,1024,16,16
4,10077,NaN,NaN,Adi,256,4,64,8192,1024,1024,...,32,128,16,8192,1024,8,16,128,128,128
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1001,62524,NaN,NaN,Sardinian Immigrants,4,2,2,16,2,8192,...,512,8192,16,8192,1024,256,128,128,128,128
1002,62526,NaN,NaN,Uintah Ute,4,4,64,16,2,2,...,32,8192,1024,8192,8192,64,16,1024,128,128
1003,62527,NaN,NaN,Hausa Zazzagawa,4,4,4,16,2,2,...,512,8192,128,8192,128,256,16,128,128,1024
1004,62528,NaN,NaN,Hausa Unspecified,1024,256,32,8192,128,8192,...,32,8192,1024,128,16,256,16,16,128,1024
